## warning : 환경 바꿔야 함.

conda activate pointnext

In [4]:
cd /data/wlsgur4011/part_assembly/src/pointnext

/data/wlsgur4011/part_assembly/src/pointnext


In [5]:
%matplotlib widget

[2023-05-22 05:26:15] [DEBU] Loaded backend module://ipympl.backend_nbagg version unknown.


In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from jhutil import show_point_clouds
import torch

# 1. ShapeNet Part Segmentation

### import and config

In [7]:
import argparse
import yaml
import os
import sys
import logging
import wandb
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import distributed as dist, multiprocessing as mp
from torch.utils.tensorboard import SummaryWriter
from torch_scatter import scatter
import torch.nn.functional as F
import warnings
import numpy as np
from sklearn.metrics import confusion_matrix
from collections import defaultdict, Counter

torch.backends.cudnn.benchmark = False
warnings.simplefilter(action='ignore', category=FutureWarning)
sys.path.insert(0, "/data/wlsgur4011/part_assembly/src/pointnext/examples")

from openpoints.models import build_model_from_cfg
from openpoints.models.layers import torch_grouping_operation, knn_point
from openpoints.loss import build_criterion_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.dataset import build_dataloader_from_cfg, get_class_weights, get_features_by_keys
from openpoints.transforms import build_transforms_from_cfg
from openpoints.utils import AverageMeter, ConfusionMatrix
from openpoints.utils import set_random_seed, save_checkpoint, load_checkpoint, resume_checkpoint, setup_logger_dist, \
    cal_model_parm_nums, Wandb, generate_exp_directory, resume_exp_directory, EasyConfig, dist_utils, find_free_port
from openpoints.models.layers import furthest_point_sample



[2023-05-22 05:26:28] [DEBU] Popen(['git', 'version'], cwd=/data/wlsgur4011/part_assembly/src/pointnext, universal_newlines=False, shell=None, istream=None)
[2023-05-22 05:26:28] [DEBU] Popen(['git', 'version'], cwd=/data/wlsgur4011/part_assembly/src/pointnext, universal_newlines=False, shell=None, istream=None)
[2023-05-22 05:26:28] [DEBU] Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
[2023-05-22 05:26:28] [DEBU] No config file found
[2023-05-22 05:26:30] [DEBU] Creating converter from 7 to 5
[2023-05-22 05:26:30] [DEBU] Creating converter from 5 to 7
[2023-05-22 05:26:30] [DEBU] Creating converter from 7 to 5
[2023-05-22 05:26:30] [DEBU] Creating converter from 5 to 7


In [8]:
import sys
sys.argv = ["examples/segmentation/main.py" ,"--cfg" ,"cfgs/shapenetpart/pointnext-s.yaml"]

In [9]:
def get_config():
    parser = argparse.ArgumentParser('ShapeNetPart Part segmentation training')
    parser.add_argument('--cfg', type=str, required=True, help='config file')
    args, opts = parser.parse_known_args()
    cfg = EasyConfig()
    cfg.load(args.cfg, recursive=True)
    cfg.update(opts)
    if cfg.seed is None:
        cfg.seed = np.random.randint(1, 10000)
    # init distributed env first, since logger depends on the dist info.
    cfg.rank, cfg.world_size, cfg.distributed, cfg.mp = dist_utils.get_dist_info(cfg)
    cfg.sync_bn = cfg.world_size > 1

    # logger
    # init log dir
    cfg.task_name = args.cfg.split('.')[-2].split('/')[-2]  # task/dataset name, \eg s3dis, modelnet40_cls
    cfg.cfg_basename = args.cfg.split('.')[-2].split('/')[-1]  # cfg_basename, \eg pointnext-xl
    tags = [
        cfg.task_name,  # task name (the folder of name under ./cfgs
        cfg.mode,
        cfg.cfg_basename,  # cfg file name
        f'ngpus{cfg.world_size}',
        f'seed{cfg.seed}',
    ]
    opt_list = [] # for checking experiment configs from logging file
    for i, opt in enumerate(opts):
        if 'rank' not in opt and 'dir' not in opt and 'root' not in opt and 'pretrain' not in opt and 'path' not in opt and 'wandb' not in opt and '/' not in opt:
            opt_list.append(opt)
    cfg.root_dir = os.path.join(cfg.root_dir, cfg.task_name)
    cfg.opts = '-'.join(opt_list)

    cfg.is_training = cfg.mode not in ['test', 'testing', 'val', 'eval', 'evaluation']

    if cfg.mode in ['resume', 'test', 'val']:
        resume_exp_directory(cfg, pretrained_path=cfg.pretrained_path)
        cfg.wandb.tags = [cfg.mode]
    else:
        generate_exp_directory(cfg, tags, additional_id=os.environ.get('MASTER_PORT', None))
        cfg.wandb.tags = tags
    os.environ["JOB_LOG_DIR"] = cfg.log_dir
    cfg_path = os.path.join(cfg.run_dir, "cfg.yaml")
    with open(cfg_path, 'w') as f:
        yaml.dump(cfg, f, indent=2)
        os.system('cp %s %s' % (args.cfg, cfg.run_dir))
    cfg.cfg_path = cfg_path
    return cfg

cfg = get_config()

launch mp with 1 GPUs, current rank: 0


### dataloader and model

In [10]:

val_loader = build_dataloader_from_cfg(cfg.batch_size,
                                       cfg.dataset,
                                       cfg.dataloader,
                                       datatransforms_cfg=cfg.datatransforms,
                                       split='val',
                                       distributed=False
                                       )
logging.info(f"length of validation dataset: {len(val_loader.dataset)}")
num_classes = val_loader.dataset.num_classes if hasattr(
    val_loader.dataset, 'num_classes') else None
if num_classes is not None:
    assert cfg.num_classes == num_classes

cfg.cls2parts = val_loader.dataset.cls2parts

if cfg.model.get('decoder_args', False):
    cfg.model.decoder_args.cls2partembed = val_loader.dataset.cls2partembed
if cfg.model.get('in_channels', None) is None:
    cfg.model.in_channels = cfg.model.encoder_args.in_channels
    
model = build_model_from_cfg(cfg.model).cuda()

data/ShapeNetPart/shapenetcore_partanno_segmentation_benchmark_v0_normal/processed/test_2048_fps.pkl load successfully
[2023-05-22 05:27:35] [INFO] length of validation dataset: 2874
[2023-05-22 05:27:35] [INFO] radius: [[0.1], [0.1], [0.25], [0.625], [1.5625]],
 nsample: [[32], [32], [32], [32], [32]]
[2023-05-22 05:27:35] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.1
nsample: 32
[2023-05-22 05:27:35] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.25
nsample: 32
[2023-05-22 05:27:35] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.625
nsample: 32
[2023-05-22 05:27:35] [INFO] NAME: ballquery
normalize_dp: True
radius: 1.5625
nsample: 32


In [11]:
train_loader = build_dataloader_from_cfg(cfg.batch_size,
                                         cfg.dataset,
                                         cfg.dataloader,
                                         datatransforms_cfg=cfg.datatransforms,
                                         split='train',
                                         distributed=False,
                                         )
len(train_loader.dataset)

13998

### RUN!!! 

In [8]:

from openpoints.dataset import get_features_by_keys
from openpoints.loss import build_criterion_from_cfg


In [9]:
criterion = build_criterion_from_cfg(cfg.criterion_args).cuda()
for data in train_loader:
    import jhutil; jhutil.jhprint(1000, data)
    batch_size, num_point, _ = data['pos'].size()
    for key in data.keys():
        data[key] = data[key].cuda(non_blocking=True)
    target = data['y']
    data['x'] = get_features_by_keys(data, cfg.feature_keys)
    import jhutil; jhutil.jhprint(1111, data['x'])
    import jhutil; jhutil.jhprint(1222, data['y'])

    logits = model(data)
    import jhutil; jhutil.jhprint(2222, logits)
    if cfg.criterion_args.NAME != 'MultiShapeCrossEntropy':
        loss = criterion(logits, target)
        import jhutil; jhutil.jhprint(3111, )
    else:
        loss = criterion(logits, target, data['cls'])
        import jhutil; jhutil.jhprint(3222, )
    import jhutil; jhutil.jhprint(3333, loss)
    
    break

1000  {
    "pos": "tensor[8, 2048, 3] n=49152 (0.2Mb) x∈[-0.968, 1.000] μ=-4.777e-06 σ=0.330",
    "x": "tensor[8, 2048, 3] n=49152 (0.2Mb) x∈[-1.000, 1.000] μ=-0.001 σ=0.540",
    "cls": "tensor[8, 1] i64 x∈[0, 15] μ=9.000 σ=6.887 [[4], [0], [15], [15], [15], [15], [0], [8]]",
    "y": "tensor[8, 2048] i64 n=16384 (0.1Mb) x∈[0, 49] μ=28.650 σ=20.130",
    "heights": "tensor[8, 2048, 1] n=16384 (64Kb) x∈[0., 0.740] μ=0.289 σ=0.217"
}
1111  "tensor[8, 7, 2048] n=114688 (0.4Mb) x∈[-1.000, 1.000] μ=0.041 σ=0.434 cuda:0"
1222  "tensor[8, 2048] i64 n=16384 (0.1Mb) x∈[0, 49] μ=28.650 σ=20.130 cuda:0"
2222  "tensor[8, 50, 2048] n=819200 (3.1Mb) x∈[-3.477, 3.222] μ=0.024 σ=0.567 grad SqueezeBackward1 cuda:0"
3111 
3333  "tensor grad MeanBackward0 cuda:0 0.335"


In [30]:

import jhutil; jhutil.jhprint(1111, data['x'][0, :,::200].v) # continuous한 feature
import jhutil; jhutil.jhprint(1222, data['y'][0, ::200].v)   # label 값

1111  "tensor[7, 11] n=77 x∈[-1.000, 1.000] μ=0.004 σ=0.467 cuda:0\ntensor([[-0.5118,  0.0872,  0.4139, -0.1361, -0.5627, -0.1265,  0.0976, -0.5295,\n         -0.2294,  0.4175, -0.2682],\n        [ 0.4132,  0.3189,  0.3202,  0.3203, -0.4667, -0.3769,  0.4153,  0.2147,\n          0.3216, -0.3956, -0.3761],\n        [ 0.3295, -0.1068,  0.0104, -0.3904, -0.3969,  0.4460, -0.0056, -0.0632,\n         -0.5192, -0.2760, -0.4664],\n        [ 0.0000,  0.0000,  0.0000,  0.0000, -1.0000, -0.0314,  0.0000, -0.6675,\n          0.0000,  0.0036,  0.0000],\n        [ 1.0000, -1.0000, -1.0000, -1.0000,  0.0000,  0.9976,  1.0000, -0.7446,\n         -1.0000,  1.0000,  0.9955],\n        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0040,  0.0623,  0.0000,  0.0000,\n          0.0000,  0.0000,  0.0952],\n        [ 0.4905,  0.4486,  0.4486,  0.4486,  0.0962,  0.1367,  0.4905,  0.4017,\n          0.4486,  0.1282,  0.1367]], device='cuda:0')"
1222  "tensor[11] i64 x∈[47, 49] μ=47.545 σ=0.688 cuda:0\ntensor([47, 47, 

In [34]:
import jhutil; jhutil.jhprint(6666, "feature_keys:", cfg.feature_keys) # 그냥  coordinate 이외에 다른 값들을 주나 봄

6666  "feature_keys:" "pos,x,heights"


# 2. S3DIS Segmentation

## import and config

In [ ]:
import __init__
import argparse
import yaml
import os
import logging
import numpy as npb
import csv
import wandb
import glob
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import distributed as dist, multiprocessing as mp
from torch.utils.tensorboard import SummaryWriter
from torch_scatter import scatter
from openpoints.utils import set_random_seed, save_checkpoint, load_checkpoint, resume_checkpoint, setup_logger_dist, \
    cal_model_parm_nums, Wandb, generate_exp_directory, resume_exp_directory, EasyConfig, dist_utils, find_free_port
from openpoints.utils import AverageMeter, ConfusionMatrix, get_mious
from openpoints.dataset import build_dataloader_from_cfg, get_features_by_keys, get_class_weights
from openpoints.dataset.data_util import voxelize
from openpoints.dataset.semantic_kitti.semantickitti import load_label_kitti, load_pc_kitti, remap_lut_read, remap_lut_write, get_semantickitti_file_list
from openpoints.transforms import build_transforms_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.loss import build_criterion_from_cfg
from openpoints.models import build_model_from_cfg
import warnings

In [5]:
import sys
sys.argv = ["examples/segmentation/main.py" ,"--cfg" ,"cfgs/s3dis/pointnext-s.yaml"]


In [2]:
def get_config():
    parser = argparse.ArgumentParser('Scene segmentation training/testing')
    parser.add_argument('--cfg', type=str, required=True, help='config file')
    parser.add_argument('--profile', action='store_true', default=False, help='set to True to profile speed')
    args, opts = parser.parse_known_args()
    cfg = EasyConfig()
    cfg.load(args.cfg, recursive=True)
    cfg.update(opts)  # overwrite the default arguments in yml
    
    if cfg.seed is None:
        cfg.seed = np.random.randint(1, 10000)
    
    # init distributed env first, since logger depends on the dist info.
    cfg.rank, cfg.world_size, cfg.distributed, cfg.mp = dist_utils.get_dist_info(cfg)
    cfg.sync_bn = cfg.world_size > 1
    
    # init log dir
    cfg.task_name = args.cfg.split('.')[-2].split('/')[-2]  # task/dataset name, \eg s3dis, modelnet40_cls
    cfg.cfg_basename = args.cfg.split('.')[-2].split('/')[-1]  # cfg_basename, \eg pointnext-xl
    tags = [
        cfg.task_name,  # task name (the folder of name under ./cfgs
        cfg.mode,
        cfg.cfg_basename,  # cfg file name
        f'ngpus{cfg.world_size}',
        f'seed{cfg.seed}',
    ]
    opt_list = []  # for checking experiment configs from logging file
    for i, opt in enumerate(opts):
        if 'rank' not in opt and 'dir' not in opt and 'root' not in opt and 'pretrain' not in opt and 'path' not in opt and 'wandb' not in opt and '/' not in opt:
            opt_list.append(opt)
    cfg.root_dir = os.path.join(cfg.root_dir, cfg.task_name)
    cfg.opts = '-'.join(opt_list)
    
    cfg.is_training = cfg.mode not in ['test', 'testing', 'val', 'eval', 'evaluation']
    if cfg.mode in ['resume', 'val', 'test']:
        resume_exp_directory(cfg, pretrained_path=cfg.pretrained_path)
        cfg.wandb.tags = [cfg.mode]
    else:
        generate_exp_directory(cfg, tags, additional_id=os.environ.get('MASTER_PORT', None))
        cfg.wandb.tags = tags
    os.environ["JOB_LOG_DIR"] = cfg.log_dir
    cfg_path = os.path.join(cfg.run_dir, "cfg.yaml")
    with open(cfg_path, 'w') as f:
        yaml.dump(cfg, f, indent=2)
        os.system('cp %s %s' % (args.cfg, cfg.run_dir))
    cfg.cfg_path = cfg_path
    
    # wandb config
    cfg.wandb.name = cfg.run_name
    
    if cfg.model.get('in_channels', None) is None:
        cfg.model.in_channels = cfg.model.encoder_args.in_channels
        
    return config

set_random_seed(cfg.seed + cfg.rank, deterministic=cfg.deterministic)
torch.backends.cudnn.enabled = True

In [ ]:
config = get_config()

## model & data_loader

In [ ]:
model = build_model_from_cfg(cfg.model).to(cfg.rank)
model_size = cal_model_parm_nums(model)

In [ ]:
val_loader = build_dataloader_from_cfg(cfg.get('val_batch_size', cfg.batch_size),
                                       cfg.dataset,
                                       cfg.dataloader,
                                       datatransforms_cfg=cfg.datatransforms,
                                       split='val',
                                       distributed=cfg.distributed
                                    )

## RUN !!!

In [ ]:
model.eval()  # set model to eval mode
cm = ConfusionMatrix(num_classes=cfg.num_classes, ignore_index=cfg.ignore_index)
pbar = tqdm(enumerate(val_loader), total=val_loader.__len__(), desc='Val')
for idx, data in pbar:
    keys = data.keys() if callable(data.keys) else data.keys
    for key in keys:
        data[key] = data[key].cuda(non_blocking=True)
    target = data['y'].squeeze(-1)
    data['x'] = get_features_by_keys(data, cfg.feature_keys)
    logits = model(data)